In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as f
import spacy
import sklearn as sl

In [11]:
import re

### I tried using logistic regression, MultiPerceptron, and tried to use CNNs and LSTMs for this task, couldn't fit the last two because of time issues and the fact that i couldn't download the gLOVe vectors perfectly due to network issues.

In [197]:
import torch.utils.data

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/keshavpc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [170]:
train = pd.read_csv("datasets/V1.4_Training.csv")#loading dataset

In [171]:
texts = np.concatenate((['"Please enable removing language code from the Dev Center "language history" For example if you ever selected "ru" and "ru-ru" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad."'],
                        train.iloc[:]['"Please enable removing language code from the Dev Center "language history" For example if you ever selected "ru" and "ru-ru" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad."'])
                       ,axis=0)

#### The following function corrects the common mishappenings of a sentence which may hinder the training

In [172]:
# Lowercase and remove whitespaces
def preprocess(texts):
    
    texts = [text.replace('"','') for text in texts]
    texts = [text.lower().strip() for text in texts]


    # Remove numbers
    texts = [re.sub(r'\d+', '', text) for text in texts]


    #Remove Punctuation
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(texts)):
        texts[i] = ' '.join(tokenizer.tokenize(texts[i]))
        
    # REMOVE STOPWORDS
    STOPWORDS = set(stopwords.words('english'))
    texts = [' '.join([word for word in text.split() if word not in STOPWORDS]) for text in texts ] 
    
    return texts 

In [173]:
texts = preprocess(texts)

In [174]:
labels = np.concatenate(([1],train.iloc[:]['1'].values),axis=0)

In [175]:
new_dataframe = pd.DataFrame({'texts':[],'labels':[],'len':[]})

In [176]:
new_dataframe['texts'] = texts
new_dataframe['labels'] = labels
# Now bucketing the data, as the texts are of different lengths.
new_dataframe['len'] = [len(text) for text in texts]

In [177]:
len(new_dataframe)

8500

In [178]:
for i in range(len(new_dataframe)):
    if(new_dataframe.loc[i].len == 0):
        new_dataframe = new_dataframe.drop(i)

In [179]:
len(new_dataframe)

8478

### The above steps removed null sentences(22) from the dataset

In [180]:
X_train = new_dataframe['texts']

In [ ]:
y_train = new_dataframe['labels']

In [157]:
test = pd.read_csv("datasets/test_labelled.csv",encoding = "ISO-8859-1", engine='python')#loading dataset

In [162]:
X_test = preprocess(test['sentence'].values)

In [265]:
X_test = preprocess(test['sentence'])

In [266]:
y_test = test['label']

In [203]:
class_a = new_dataframe[new_dataframe.labels == 1]

In [195]:
class_b = new_dataframe[new_dataframe.labels == 0]

In [205]:
class_a = [class_a,class_a,class_a]

In [207]:
class_a_final = pd.concat(class_a)

In [358]:
len(class_a_final)

6255

In [209]:
merging = [class_a_final,class_b]

In [210]:
new_dataframe = pd.concat(merging)

In [213]:
length = len(new_dataframe)

In [212]:
#Shuffling the data
from sklearn.utils import shuffle
new_dataframe = shuffle(new_dataframe)

## Above steps basically dealing with the class imbalance problem in the dataset, as the labels had the ratio of 3:1, upscales the class with less samples.

In [372]:
three_trains = [new_dataframe[:length//3],new_dataframe[length//3:2*length//3],new_dataframe[2*length//3:]]

In [373]:
X_train = []
y_train = []

In [374]:
for i in three_trains:
    X_train.append(i['texts'].values)
    y_train.append(i['labels'].values)

In [375]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,10),token_pattern='(\S+)')

In [295]:
X_train_tfidf = []

In [400]:
model = LogisticRegression(penalty='l2', C=1.0,solver='liblinear')

In [401]:
predictions = []

### Runing logistic regression over the three segmented datasets.

In [402]:
for i in range(3):
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train[i]).toarray()
    X_test_tfidf = tfidf_vectorizer.transform(list(X_test)).toarray()
    model.fit(X_train_tfidf, y_train[i])
    predictions.append(model.predict(X_test_tfidf))
    

## Training over the three seperately to get an average of the performance in above steps

In [379]:
f1_scores = []

In [380]:
accuracies = []

In [381]:
for y_predictions in predictions:
    accuracies.append(sum(y_test == y_predictions)*100/len(y_test))
    f1_scores.append(f1_score(y_test, y_predictions, average='macro')) 

In [382]:
accuracies

[70.27027027027027, 70.4391891891892, 71.45270270270271]

In [383]:
f1_scores

[0.7026891293430124, 0.7037757158991809, 0.7138403365911281]

##### ~ 70% accuracy using logistic regression

In [139]:
from sklearn.neural_network import MLPClassifier

In [359]:
clf = MLPClassifier(solver='adam', alpha=1e-5,learning_rate_init=3e-2,learning_rate='adaptive',random_state=40,
                   early_stopping=True,hidden_layer_sizes=(200,32,))

In [360]:
#Took around 10 minutes
predictions_mlp = []

In [361]:
for i in range(3):
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train[i]).toarray()
    X_test_tfidf = tfidf_vectorizer.transform(list(X_test)).toarray()
    clf.fit(X_train_tfidf, y_train[i])
    predictions_mlp.append(clf.predict(X_test_tfidf))
    
  

I Tried using LSTMs and CNN models for this task however i failed to catch errors which prevented it from showing better scores in the given time frame.
#### The bad performance is most probably due to the loose tokenization by tf-idf, i wanted to use predefined gLOve ones but I couldn't download them due to internet issues. 

In [362]:
accuracies_mlp = []
f1_scores_mlp = []

In [363]:
for y_predictions in predictions_mlp:
    accuracies_mlp.append(sum(y_test == y_predictions)*100/len(y_test))
    f1_scores_mlp.append(f1_score(y_test, y_predictions, average='macro')) 

In [364]:
accuracies_mlp

[60.979729729729726, 63.3445945945946, 64.86486486486487]

In [357]:
f1_scores_mlp

[0.6388872903886658, 0.6545306629333962, 0.6402779286274118]

### Trained using the default perceptron by sklearn, not a significent boost in performance, indicating that the tf-idf vectorization was not good.

#### Now importing the actual test set

In [394]:
test = pd.read_csv('datasets/SubtaskA_EvaluationData.csv')

In [ ]:
This would enable live traffic aware apps.

In [397]:
X_test_final = np.concatenate((['This would enable live traffic aware apps.'],
                        test.iloc[:]['This would enable live traffic aware apps.'])
                       ,axis=0)

In [405]:
X_test_tfidf = tfidf_vectorizer.transform(list(preprocess(X_test_final))).toarray()
y_predicted = model.predict(X_test_tfidf)

In [410]:
out_dataframe = pd.DataFrame({'texts':X_test_final,'labels':y_predicted})

In [412]:
out_dataframe.to_csv('keshav_vinayak_jha.csv',index=False)